In [ ]:
!pip install transformers
!pip install finvizfinance

#### Average Sentiment of Stock Market News

This notebook showcases a simple way to calculate the news sentiment for the stock market. It does this as follows:
1. Crawl the most recent news titles using finvizfinance.
2. Classify the sentiment of each title using SiEBERT.
3. If the title has a positive sentiment, the score is 1, else 0.
4. Take the average of all scores.

A score of 0.5 would mean that 50% of the news titles have a positive sentiment.

#### Trading Strategy: Inverse Sentiment Strategy

1. Buy if the score is low: <0.4
2. Sell if the score is high: >0.8

In [ ]:
from transformers import pipeline
sentiment_analysis = pipeline("sentiment-analysis",
                              model="siebert/sentiment-roberta-large-english",
                              device=0)
print(sentiment_analysis("I love this!"))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cuda:0


[{'label': 'POSITIVE', 'score': 0.9988656044006348}]


In [ ]:
import os
from datetime import datetime
from finvizfinance.news import News
import pandas as pd

def sentiment_score(text):
    # Return 1 if text (str) has a positive sentiment, else 0.
    return sentiment_analysis(text)[0]['label'] == "POSITIVE"

def news_avg_sentiment(all_news):
    # Assuming all_news['news'] is a DataFrame with a column 'title'
    news_df = pd.DataFrame(all_news['news'])
    
    # Apply sentiment analysis to each title
    news_df['sentiment_score'] = news_df['Title'].apply(sentiment_score)
    
    # Compute average sentiment score
    average_sentiment = news_df['sentiment_score'].mean()

    return average_sentiment

def save_sentiment_to_csv(average_sentiment, csv_file):
    # Create a DataFrame with current timestamp and sentiment
    now = datetime.now()
    timestamp = now.strftime("%Y-%m-%d %H:%M:%S")
    new_entry = pd.DataFrame({
        'timestamp': [timestamp],
        'average_sentiment': [average_sentiment]
    })

    # If file exists, append without writing header
    if os.path.exists(csv_file):
        new_entry.to_csv(csv_file, mode='a', header=False, index=False)
    else:
        # Create new file with header
        new_entry.to_csv(csv_file, index=False)

In [ ]:
# Crawl dataframe of news from Finviz
fnews = News()
all_news = fnews.get_news()

# Calculate sentiment
average_sentiment = news_avg_sentiment(all_news)
print(f"Average sentiment: {average_sentiment:.4f}")

# Save sentiment
save_sentiment_to_csv(average_sentiment, "finvizsentimentdata.csv")